In [329]:
import pandas as pd
import numpy as np
import string

In [330]:
dat=pd.read_excel("./AuthorList.xlsx",engine='openpyxl').set_index('id')

In [331]:
#FirstAuthorIDs=[51,79,105,13,21,32,71,74,101]
FirstAuthorIDs=[]

In [332]:
# Put the author list and list of institutions in the correct order

OrderOfAuthors=[]
for i in FirstAuthorIDs:
    OrderOfAuthors.append(i)
for i in dat.index:
    if i not in FirstAuthorIDs:
        OrderOfAuthors.append(i)

UniqueInstitutions=dat.loc[OrderOfAuthors].Institution.unique()
Addresses=[]

for u in UniqueInstitutions:
    Addresses.append(dat[dat.Institution==u].iloc[0].Address)

def InstitutionID(Name):
    return str(np.where(UniqueInstitutions==Name)[0][0])

UniqueNotes=dat.loc[OrderOfAuthors].Footnote.unique()

def NoteID(Name):
    return str(num2alpha[np.where(UniqueNotes==Name)[0][0]+1])


num2alpha = dict(zip(range(1, 27), string.ascii_lowercase))

In [333]:
# Function to make the author list for IOP journals (JCAP, JHEP, JINST...)

def MakeIOPAuthorList():
    OrderOfAuthors=GetOrderOfAuthors(FirstAuthorIDs)
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        if(len(str(entry.Footnote))>5):
            authorlist+=("\\author["+InstitutionID(entry.Institution)+","+NoteID(entry.Footnote)+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n")
            authorlist+=('\\note['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
        else:
            authorlist+=("\\author["+InstitutionID(entry.Institution)+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\affiliation[" + InstitutionID(UniqueInstitutions[i]) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+Addresses[i]+"}\n"
        authorlist+=("%\n")
    return authorlist

In [334]:
# Function to make the author list for Elsevier journals (NIM...)

def MakeElsevierAuthorList():
    OrderOfAuthors=GetOrderOfAuthors(FirstAuthorIDs)
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        if(len(str(entry.Footnote))>5):
            authorlist+=("\\author["+InstitutionID(entry.Institution)+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+"\corref{"+NoteID(entry.Footnote)+"}}\n")
            authorlist+=('\\cortext['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
        else:
            authorlist+=("\\author["+InstitutionID(entry.Institution)+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+"}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\address[" + InstitutionID(UniqueInstitutions[i]) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+ Addresses[i]+"}\n"
        authorlist+=("%\n")
    return authorlist

In [335]:
# Function to make the author list for APS journals (PRD, PRL...)

def MakeAPSAuthorList():
    OrderOfAuthors=GetOrderOfAuthors(FirstAuthorIDs)
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        authorlist+="\\author{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n"
        authorlist+="\\affiliation{"+entry.Institution.replace('\\\\','\\\\\n')+entry.Address+"}\n"
        if(len(str(entry.Footnote))>5):
            authorlist+=('\\thanks{' + entry.Footnote.strip() + '}\n')
        authorlist+=("%\n")
    return authorlist

In [336]:
# Function to make the author list for arXiv

def MakeArXivAuthorList():
    OrderOfAuthors=GetOrderOfAuthors(FirstAuthorIDs)
    authorlist=str("NEXT Collaboration: ")
    for a in OrderOfAuthors[:-1]:
        entry=dat.loc[a]
        authorlist+=entry.Initial.strip()+" "+entry.LastName.strip()+", "
    entry=dat.loc[OrderOfAuthors[-1]]
    authorlist=authorlist[:-2]+" and " + entry.Initial.strip()+" "+entry.LastName.strip()
    return authorlist

In [337]:
# Make author lists and store to .tex

f=open("authors_elsevier.tex",'w')
f.write(MakeElsevierAuthorList())
f.close()

f=open("authors_iop.tex",'w')
f.write(MakeIOPAuthorList())
f.close()

f=open("authors_aps.tex",'w')
f.write(MakeAPSAuthorList())
f.close()

f=open("authors_arxiv.tex",'w')
f.write(MakeArXivAuthorList())
f.close()